<a href="https://colab.research.google.com/github/HamsalekhaYerra/Hamsalekha_INFO_5731_Spring2022/blob/main/In_class_exercise_02_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third In-class-exercise (02/08/2022, 40 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):


Anomaly detection is a critical task for financial market, investors, and regulatory authorities, where conventional methods employ rule-based models. With the use of python it becomes more promising to detect anomalous trading behaviors from data.

Hence, our task is to detect anamolies from share price data.

The data recquired is historical share price data of any company.

The attributes we need are Date, Open share Price, High, Low, Closing share price and Volume.

To scrape the share price data of any company, we can use the Yahoo finance website to get the historical share price data.

By entering the ticker symbol, yahoo finance gives us the corresponding company's share price data, which we can import into python and export into a csv file and do our further analysis.

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [4]:
#Pip install yfinance
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
ticker = yf.Ticker("FB") #we can enter any ticker, here for reference I have taken FB's share price data
print(ticker.history(period="max"))

                  Open        High  ...  Dividends  Stock Splits
Date                                ...                         
2012-05-18   42.049999   45.000000  ...          0             0
2012-05-21   36.529999   36.660000  ...          0             0
2012-05-22   32.610001   33.590000  ...          0             0
2012-05-23   31.370001   32.500000  ...          0             0
2012-05-24   32.950001   33.209999  ...          0             0
...                ...         ...  ...        ...           ...
2022-02-07  237.699997  238.300003  ...          0             0
2022-02-08  220.850006  225.770004  ...          0             0
2022-02-09  224.199997  233.369995  ...          0             0
2022-02-10  228.270004  235.000000  ...          0             0
2022-02-11  228.460007  230.419998  ...          0             0

[2451 rows x 7 columns]


Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2012-2022).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [5]:
!pip install requests --upgrade
import requests

In [6]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
url = 'https://scholar.google.com/scholar?q=information+retrieval&hl=en&as_sdt=0%2C5&as_ylo=2012&as_yhi=2022'
response=requests.get(url,headers=headers)

In [8]:
# importing beautifulsoup
!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [9]:
# function for the getting inforamtion of the web page
def get_paperinfo(paper_url):
    response=requests.get(url,headers=headers)
    try:
        if response.status_code != 200:
            print('Status code:', response.status_code)
    
    except: print('Failed to fetch web page ')
    paper_doc = BeautifulSoup(response.text,'html.parser')
    return paper_doc

In [10]:
# function for the extracting information of the tags
def get_tags(doc):
    paper_tag = doc.select('[data-lid]')
    cite_tag = doc.select('[title=Cite] + a')
    link_tag = doc.find_all('h3',{"class" : "gs_rt"})
    author_tag = doc.find_all("div", {"class": "gs_a"})
    abstract_tag = doc.find_all("div",{"class":"gs_rs"})
    return paper_tag,cite_tag,link_tag,author_tag,abstract_tag

In [11]:
# function to return the title of the paper
def get_papertitle(paper_tag):
    paper_names = []
    for tag in paper_tag:
        paper_names.append(tag.select('h3')[0].get_text())
    return paper_names

In [12]:
# function for the getting author , year and publication information
import re
def get_author_year_publi_info(authors_tag):
    years = []
    publication = []
    authors = []
    for i in range(len(authors_tag)):
        authortag_text = (authors_tag[i].text).split()
        year = int(re.search(r'\d+', authors_tag[i].text).group())
        years.append(year)
        publication.append(authortag_text[-1])
        author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
        authors.append(author)
    return years , publication, authors

In [13]:
# function for getting abstract
def get_abstract_text(abstract_tag):
    abstract = []
    for i in range(len(abstract_tag)):
        abstracttag_text = (abstract_tag[i].text)
        abstract.append(abstracttag_text)
    return abstract

In [14]:
# creating final repository
import pandas as pd
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Publication' : [],
                    'Abstract' : []}

# adding information in repository
def add_in_paper_repo(papername,year,author,publi,abstr):
    paper_repos_dict['Paper Title'].extend(papername)
    paper_repos_dict['Year'].extend(year)
    paper_repos_dict['Author'].extend(author)
    paper_repos_dict['Publication'].extend(publi)
    paper_repos_dict['Abstract'].extend(abstr)
    return pd.DataFrame(paper_repos_dict)

In [15]:
# getting final attributes
import time
for i in range (0,110,10):
    url = "https://scholar.google.com/scholar?q=information+retrieval&hl=en&as_sdt=0%2C5&as_ylo=2012&as_yhi=2022".format(i)
    doc = get_paperinfo(url)
    paper_tag,cite_tag,link_tag,author_tag,abstract_tag = get_tags(doc)
    papername = get_papertitle(paper_tag)
    year , publication , author = get_author_year_publi_info(author_tag)
    abstract = get_abstract_text(abstract_tag)
    final = add_in_paper_repo(papername,year,author,publication,abstract)
    time.sleep(30)

Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429
Status code: 429


In [16]:
# creating final table
final

,Paper Title,Year,Author,Publication,Abstract


In [17]:
# creating csv file 
final.to_csv('information_retrieval_reserachpapers.csv', sep=',', index=False,header=True)

Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data. 

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text 

In [2]:
# You code here (Please add comments in the code):
!pip install tweepy
import pandas as pd
import tweepy as tp
import pandas as pd
import csv
import re 
import string

consumer_key= '2Em7SxlX9jPMfL4x97r3zMO0x'
consumer_secret= 'sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd'
access_key= '1439767876962029572-uUMt8oWRyzj9ilE5zk4uYbL93sCMPT'
access_secret= 'oydIGymn9bS767FVEMawE9GyGAnMmBJfaY2XXKmHnmliF'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tp.API(auth,wait_on_rate_limit=True)
 
csvFile = open('file-name', 'a')
csvWriter = csv.writer(csvFile)
 
search_words = "#COVID19"      # enter your words
new_search = search_words + " -filter:retweets"
csvWriter.writerow(["Created At","Text","User","Location"])
for tweet in tp.Cursor(api.search_tweets,q=new_search,count=100,
                           lang="en",
                           since_id=0).items():
    
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'),tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8')])
    data=pd.read_csv('file-name')
    data.head(1000)




AttributeError: ignored